In [137]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [138]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [139]:
websites = [
    'ncbi.nlm.nih.gov', 'elsevier.com', 'wiley.com', 'springer.com', 'physiology.org', 'oup.com', 
    'cambridge.org', 'karger.com', 'lww.com', 'nature.com', 'science.org', 'tandfonline.com', 
    'sagepub.com', 'jamanetwork.com', 'neurology.org', 'biorxiv.org', 'royalsocietypublishing.org', 
    'psycnet.apa.org', 'arvojournals.org', 'jstage.jst.go.jp', 'psychiatryonline.org', 'europepmc.org', 
    'mit.edu', 'thejns.org', 'annualreviews.org', 'snmjournals.org', 'aspetjournals.org', 'elibrary.ru', 
    'books.google.de', 'architalbiol.org', 'ahajournals.org', 'liebertpub.com', 'acs.org', 'degruyter.com', 
    'worldscientific.com', 'iospress.com', 'asahq.org', 'thieme-connect.de', 'neurologia.com', 'mpg.de', 
    'opg.optica.org', 'mcgill.ca', 'rbojournal.org', 'taylorfrancis.com', 'ekja.org', 'www.imrpress.com', 
    'theses.fr', 'ieee.org', 'ssrn.com', 'open.bu.edu', 'journals.biologists.com', 'aip.org', 'lib.wfu.edu', 
    'literatumonline.com', 'scholarpedia.org', 'isho.jp', 'mirasmart.com', 'microbiologyresearch.org', 
    'aps.org', 'ujms.net', 'www.ingentaconnect.com', 'symposium.cshlp.org', 'cabdirect.org', 'ajtmh.org'
]

In [141]:
# nature.com
def func_nature_com(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "introduction": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome()

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(5)
            wait = WebDriverWait(driver, 30)
            wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All Cookies')]"))).click()
            time.sleep(2)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi, pmid, pmcid
    try:
        doi = driver.find_element(By.XPATH, "//div[contains(@class, 'ej-journal-info')]").text.split("DOI: ")[1]
        doi = doi.strip()
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.XPATH, "//h1[@class='ejp-article-title']").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='ejp-article-text-abstract']").find_elements(By.TAG_NAME, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    # try:
    #     keywords = ""
    #     elems = driver.find_element(By.XPATH, "//div[@class='ejp-article-text-abstract']").find_elements(By.TAG_NAME, 'a')
    #     for elem in elems:
    #         keywords = keywords + elem.text + "; "
    #     keywords = keywords.strip()
    # except:
    #     keywords = np.nan
    keywords = np.nan
    
    # # introduction
    # try:
    #     intro = ""
    #     elements = driver.find_elements(By.TAG_NAME, "h2")
    #     for element in elements:
    #         if "Introduction" in element.text:
    #             ele_paren = element.find_element(By.XPATH, "..")
    #             intros = ele_paren.find_elements(By.TAG_NAME, "p")
    #             for intro_ele in intros:
    #                 intro = intro + intro_ele.text + " "
    #             break
    #     intro = intro.strip()
    # except:
    #     intro = np.nan
    intro = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//button[contains(text(), 'PDF')]").click()
    #     driver.switch_to_window(driver.window_handles[-1])
    #     pdf_link = driver.current_url
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# # url = 
# # url = 
# # url = 
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------

In [142]:
# science.org

In [143]:
# tandfonline.com

In [144]:
# sagepub.com

In [145]:
# jamanetwork.com

In [146]:
# neurology.org

In [147]:
# biorxiv.org